In [1]:
# Import modules
from bs4 import BeautifulSoup
import requests
from splinter import Browser
from selenium import webdriver
import pandas as pd
import time
from secret import username, password
from sqlalchemy import create_engine

In [2]:
# URL of page to be scraped
url = 'https://en.wikipedia.org/wiki/List_of_highest-grossing_films'

In [3]:
# https://splinter.readthedocs.io/en/latest/drivers/chrome.html
!which chromedriver

/usr/local/bin/chromedriver


In [4]:
executable_path = {'executable_path': '/usr/local/bin/chromedriver'}
browser = Browser('chrome', **executable_path, headless=False)

In [5]:
browser.visit(url)

In [6]:
tables = pd.read_html(url)

In [7]:
# tables

In [8]:
table_df = tables[0]

In [9]:
table_df.set_index('Rank')
table_df.head()

,Rank,Peak,Title,Worldwide gross,Year,Reference(s)
0,1,1,Avengers: Endgame,"$2,797,800,564",2019,[# 1][# 2]
1,2,1,Avatar,"$2,790,439,000",2009,[# 3][# 4]
2,3,1,Titanic,"$2,194,439,542",1997,[# 5][# 6]
3,4,3,Star Wars: The Force Awakens,"$2,068,223,624",2015,[# 7][# 8]
4,5,4,Avengers: Infinity War,"$2,048,359,754",2018,[# 9][# 10]


In [10]:
table_df.drop(columns=['Peak', 'Year', 'Reference(s)'], inplace=True)

In [11]:
table_df = table_df.reset_index()

In [12]:
ww_gross_series = table_df['Worldwide gross']

In [13]:
len(ww_gross_series)

50

In [14]:
type(ww_gross_series)

pandas.core.series.Series

In [15]:
list = []
for i in ww_gross_series:
    print(i.split('$')[-1])
    list.append(i.split('$')[-1]) 

2,797,800,564
2,790,439,000
2,194,439,542
2,068,223,624
2,048,359,754
1,671,713,208
1,656,943,394
1,518,812,988
1,516,045,911
1,450,026,933
1,402,805,868
1,346,913,161
1,341,932,398
1,332,539,889
1,309,484,461
1,290,000,000
1,263,521,126
1,242,805,359
1,238,764,765
1,214,811,252
1,159,398,397
1,153,304,495
1,148,161,807
1,142,219,401
1,131,927,996
1,128,274,794
1,123,794,079
1,108,561,013
1,104,054,072
1,084,939,099
1,074,251,311
1,074,144,248
1,073,394,593
1,066,969,703
1,066,179,725
1,056,057,273
1,050,693,953
1,045,713,802
1,034,799,409
1,029,939,903
1,028,570,889
1,027,044,677
1,025,467,110
1,023,784,195
1,021,103,568
1,004,934,033
978,087,613
976,920,103
970,761,885
968,483,777


In [16]:
split_list = []
for i in list:
    i = i.split(',')
    split_list.append(''.join(i)) 

In [17]:
merged_list = []
for i in split_list:
    print(''.join(i))
    merged_list.append(''.join(i)) 

2797800564
2790439000
2194439542
2068223624
2048359754
1671713208
1656943394
1518812988
1516045911
1450026933
1402805868
1346913161
1341932398
1332539889
1309484461
1290000000
1263521126
1242805359
1238764765
1214811252
1159398397
1153304495
1148161807
1142219401
1131927996
1128274794
1123794079
1108561013
1104054072
1084939099
1074251311
1074144248
1073394593
1066969703
1066179725
1056057273
1050693953
1045713802
1034799409
1029939903
1028570889
1027044677
1025467110
1023784195
1021103568
1004934033
978087613
976920103
970761885
968483777


In [18]:
len(merged_list)

50

In [20]:
merged_list

['2797800564',
 '2790439000',
 '2194439542',
 '2068223624',
 '2048359754',
 '1671713208',
 '1656943394',
 '1518812988',
 '1516045911',
 '1450026933',
 '1402805868',
 '1346913161',
 '1341932398',
 '1332539889',
 '1309484461',
 '1290000000',
 '1263521126',
 '1242805359',
 '1238764765',
 '1214811252',
 '1159398397',
 '1153304495',
 '1148161807',
 '1142219401',
 '1131927996',
 '1128274794',
 '1123794079',
 '1108561013',
 '1104054072',
 '1084939099',
 '1074251311',
 '1074144248',
 '1073394593',
 '1066969703',
 '1066179725',
 '1056057273',
 '1050693953',
 '1045713802',
 '1034799409',
 '1029939903',
 '1028570889',
 '1027044677',
 '1025467110',
 '1023784195',
 '1021103568',
 '1004934033',
 '978087613',
 '976920103',
 '970761885',
 '968483777']

In [21]:
# Rename the columns to be able to merge the data 
table_df.rename(columns={"Title": "name", "Rank": "rank", "Worldwide gross":"worldwide gross"}, inplace=True)

In [22]:
# Change revenue column from string to big integer
table_df['ww_gross'] = merged_list

In [23]:
table_df.head()

,index,rank,name,worldwide gross,ww_gross
0,0,1,Avengers: Endgame,"$2,797,800,564",2797800564
1,1,2,Avatar,"$2,790,439,000",2790439000
2,2,3,Titanic,"$2,194,439,542",2194439542
3,3,4,Star Wars: The Force Awakens,"$2,068,223,624",2068223624
4,4,5,Avengers: Infinity War,"$2,048,359,754",2048359754


### Connect to local database

In [24]:
connection_string = f'{username}:{password}@localhost:5432/movies_db'
engine = create_engine(f'postgresql://{connection_string}')

In [25]:
engine.table_names()

['domestic_revenue']

### Use pandas to load csv converted DataFrame into database

In [26]:
table_df.to_sql(name='international', con=engine, if_exists='append', index=False)

### Confirm data has been added by querying the international table

In [27]:
pd.read_sql_query('select * from international', con=engine).head()

,index,rank,name,worldwide gross,ww_gross
0,0,1,Avengers: Endgame,"$2,797,800,564",2797800564
1,1,2,Avatar,"$2,790,439,000",2790439000
2,2,3,Titanic,"$2,194,439,542",2194439542
3,3,4,Star Wars: The Force Awakens,"$2,068,223,624",2068223624
4,4,5,Avengers: Infinity War,"$2,048,359,754",2048359754


In [28]:
pd.read_sql_query('select * from domestic_revenue', con=engine).head()

,id,gross,name
0,229,52287414,Stand by Me
1,230,70136369,Ferris Bueller's Day Off
2,231,179800601,Top Gun
3,232,85160248,Aliens
4,233,18564613,Flight of the Navigator


### Confirm data has been added by querying the customer_name table